In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [2]:
train_dir = 'dataset/train'
test_dir = 'dataset/test'

In [3]:
input_shape = (128,128,3)
batch_size = 32

In [4]:
data_generator = ImageDataGenerator(rescale=1./255)
train_generator = data_generator.flow_from_directory(
    train_dir, 
    target_size = (input_shape[0],input_shape[1]),
    batch_size = batch_size,
    class_mode = 'categorical'
)
test_generator = data_generator.flow_from_directory(
    test_dir, 
    target_size = (input_shape[0],input_shape[1]),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 4010 images belonging to 2 classes.
Found 3543 images belonging to 2 classes.


In [5]:
cnn_model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
     Conv2D(256, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Dense(128),
    Dense(256),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
train_features = cnn_model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  
train_labels = train_generator.classes

C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


126/126 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step


In [7]:
test_features = cnn_model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

 54/111 ━━━━━━━━━━━━━━━━━━━━ 9s 166ms/step

C:\Users\user\anaconda3\Lib\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


111/111 ━━━━━━━━━━━━━━━━━━━━ 19s 174ms/step


In [8]:
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(train_features, train_labels)

SVC(kernel='linear')

In [9]:
test_predictions = svm_classifier.predict(test_features)

In [10]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 51.31%


In [11]:
cnn_model.save('cnn_Feature_Extractor.h5')

In [12]:
joblib.dump(svm_classifier, 'svm_classifier.pkl')

['svm_classifier.pkl']

In [14]:
from tensorflow.keras.preprocessing import image
# Load the saved models
cnn_model = load_model('cnn_Feature_Extractor.h5')
knn_classifier = joblib.load('svm_classifier.pkl')

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128)) 
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  
    img_array /= 255.0  

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for SVM

    # Classify the features with the KNN model
    prediction = knn_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'with mask', 1: 'without mask'}  
    result = class_labels[prediction[0]]
    return result

# Test the function with a new image
img_path = 'dataset/test/without_mask/without_mask_2476.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
The image is classified as: without mask
